python example/env/test.py --prefix model/3dbox/3dbox --epoch 20 --image data/test_data/GTA_00001.jpg --gpu 6

In [1]:
import argparse
import os
import cv2
import mxnet as mx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from rcnn.config import config
from rcnn.core.detector import Detector
from rcnn.symbol import get_vgg_test
from rcnn.processing.image_processing import resize, transform
from rcnn.processing.nms import nms
from rcnn.core.tester import vis_all_detection
from rcnn.utils.load_model import load_param

from rcnn.processing.bbox_transform import bbox_pred, clip_boxes

CLASSES = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')

def get_net(prefix, epoch, ctx):
    args, auxs = load_param(prefix, epoch, convert=True, ctx=ctx)
    #sym = get_vgg_test(num_classes=4, num_anchors=12)
    sym = get_vgg_test()

    #a = mx.viz.plot_network(sym, shape={"data":(1, 1, 600, 1000),  "im_info":(3), 
    #                                    "gt_boxes":(1,1,5), "gt_dims":(1,1,3), "gt_angles":(1,1,1), "gt_confs":(1,1,1)}, 
    #                        node_attrs={"shape":'rect',"fixedsize":'false'})
    #a.view()

    detector = Detector(sym, ctx, args, auxs)
    return detector

prefix='model/3dbox/3dbox'
epoch=20
ctx=mx.gpu(6)
config.TEST.HAS_RPN = True

detector = get_net(prefix, epoch, ctx)

image_name='data/test_data/GTA_00001.jpg'
assert os.path.exists(image_name), image_name + ' not found'
im = cv2.imread(image_name)
im_array, im_scale = resize(im, 360, 124200) 
im_array = transform(im_array, config.PIXEL_MEANS)
im_info = np.array([[im_array.shape[2], im_array.shape[3], im_scale]], dtype=np.float32)

In [2]:
scores, boxes = detector.im_detect(im_array, im_info)

to executor
forward done
save output done


forward done


In [26]:
output_dict.items()

[('bbox_pred_reshape_output', <NDArray 1x300x84 @gpu(6)>),
 ('cls_prob_reshape_output', <NDArray 1x300x21 @gpu(6)>),
 ('dim_pred_reshape_output', <NDArray 1x300x3 @gpu(6)>),
 ('angle_pred_reshape_output', <NDArray 1x300x16 @gpu(6)>),
 ('rois_output', <NDArray 300x5 @gpu(6)>),
 ('conf_pred_reshape_output', <NDArray 1x300x1 @gpu(6)>)]